In [ ]:
#Extracting data from Open Weather api. We need to first determine (from our airbnb csv's)
# what cities to extract from OpenWeather api, because we can't extract the whole planet of cities

In [ ]:
#Extracting cities from our New Zealand spreadsheet

In [13]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import weather_api_key

#Import Data files:
new_zealand_df = pd.read_csv("airbb_data/new_zealand.csv")

#Grab lat and long b/c we will use them to find the cities later
new_zealand = new_zealand_df[['latitude', 'longitude']]

In [14]:
new_zealand.head()

,latitude,longitude
0,-38.13493,178.31671
1,-39.45078,173.84357
2,-38.99097,175.80902
3,-38.73298,177.58658
4,-37.67498,176.23216


In [15]:
#Using the citipy package to find the nearest city to the lat and long provided
new_zealand['city'] = new_zealand.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).city_name, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
#Using the citipy package to find the nearest city to the lat and long provided
new_zealand['country'] = new_zealand.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).country_code, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
#There are new zealand listings that aren't actually in new zealand
new_zealand = new_zealand[new_zealand.country == 'nz']
new_zealand.head()

,latitude,longitude,city,country
0,-38.13493,178.31671,tolaga bay,nz
1,-39.45078,173.84357,opunake,nz
2,-38.99097,175.80902,turangi,nz
3,-38.73298,177.58658,manutuke,nz
4,-37.67498,176.23216,tauranga,nz


In [19]:
#Grouping by city to remove duplicates

new_zealand_cities = new_zealand[['city']].groupby(['city'])
new_zealand_cities_df = new_zealand_cities.sum()
new_zealand_cities_df.reset_index(level=0, inplace=True)
new_zealand_cities_df

,city
0,ahipara
1,amberley
2,athenree
3,auckland
4,balclutha
...,...
145,whangarei
146,whitianga
147,winton
148,woodend


In [20]:
#Grouping by city to remove duplicates

new_zealand_countries = new_zealand[['country']].groupby(['country'])
new_zealand_countries_df = new_zealand_countries.sum()
new_zealand_countries_df.reset_index(level=0, inplace=True)
new_zealand_countries_df

,country
0,nz


In [21]:
#Transforming our city series into a city list to use with the api

nz_city = new_zealand_cities_df['city']
#nz_city
cities = nz_city.values.tolist()
#cities

In [28]:
#Transforming our city series into a city list to use with the api

nz_country = new_zealand_countries_df['country']
#nz_city
country = nz_country[0]


In [29]:
#Open weather information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"



#field_names
#empty lists for the fields of our future city weather csv
city_list = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country_list = []

for city in cities:
    
    #Get the city's number
    city_no = cities.index(city) + 1
    
    try:
        query_url = f"{url}appid={weather_api_key}&q={city},{country}&units={units}"
        
        

        # Get weather data
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()

        #Put the city's weather into lists
        city_list.append(weather_json["name"])
        lat.append(weather_json["coord"]["lat"])
        lng.append(weather_json["coord"]["lon"])
        max_temp.append(weather_json["main"]["temp_max"])
        humidity.append(weather_json["main"]["humidity"])
        cloudiness.append(weather_json["clouds"]["all"])
        wind_speed.append(weather_json["wind"]["speed"])
        country_list.append(weather_json["sys"]["country"])
        
        print(f"Getting {city}\'s weather information(City #{city_no})." )
    except KeyError:
        print(f"Can't find {city} (City #{city_no}) .")

Getting ahipara's weather information(City #1).
Getting amberley's weather information(City #2).
Getting athenree's weather information(City #3).
Getting auckland's weather information(City #4).
Getting balclutha's weather information(City #5).
Getting bluff's weather information(City #6).
Getting bombay's weather information(City #7).
Getting brightwater's weather information(City #8).
Getting bulls's weather information(City #9).
Getting burnham's weather information(City #10).
Getting christchurch's weather information(City #11).
Getting clyde's weather information(City #12).
Getting coromandel's weather information(City #13).
Getting darfield's weather information(City #14).
Getting dargaville's weather information(City #15).
Getting dobson's weather information(City #16).
Getting dunedin's weather information(City #17).
Getting edendale's weather information(City #18).
Getting edgecumbe's weather information(City #19).
Getting eltham's weather information(City #20).
Can't find fai

In [33]:
#Putting city results into a dictionary
city_dict = {'City': city_list, 'Lat': lat, 'Lng': lng, "Max Temp": max_temp, "Humidity": humidity, "Cloudiness": cloudiness, "Wind Speed": wind_speed, "Country": country_list}


In [31]:
#loading New Zealand

In [35]:
#Putting city results into a csv

zd = zip(*city_dict.values())
#, encoding='utf-8'

with open("weather_data/cities.csv", 'w', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',', lineterminator = '\n')
    writer.writerow(city_dict.keys())
    writer.writerows(zd)


In [ ]:
#Extract for Barcelona

In [36]:
barcelona_df = pd.read_csv("airbb_data/barcelona-airbb.csv")

barcelona = barcelona_df[['latitude', 'longitude']]

In [37]:
barcelona.head()

,latitude,longitude
0,41.37973,2.17631
1,41.41291,2.22063
2,41.40566,2.17015
3,41.37916,2.17535
4,41.37978,2.17623


In [38]:
#Using the citipy package to find the nearest city to the lat and long provided
barcelona['city'] = barcelona.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).city_name, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
#Using the citipy package to find the nearest city to the lat and long provided
barcelona['country'] = barcelona.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).country_code, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
barcelona.head()

,latitude,longitude,city,country
0,41.37973,2.17631,barcelona,es
1,41.41291,2.22063,badalona,es
2,41.40566,2.17015,barcelona,es
3,41.37916,2.17535,barcelona,es
4,41.37978,2.17623,barcelona,es


In [41]:
#Grouping by city to remove duplicates

barcelona_cities = barcelona[['city']].groupby(['city'])
barcelona_cities_df = barcelona_cities.sum()
barcelona_cities_df.reset_index(level=0, inplace=True)
barcelona_cities_df

,city
0,badalona
1,barcelona
2,el prat de llobregat
3,ripollet
4,sant cugat del valles
5,sant just desvern


In [42]:
#Grouping by city to remove duplicates

barcelona_countries = barcelona[['country']].groupby(['country'])
barcelona_countries_df = barcelona_countries.sum()
barcelona_countries_df.reset_index(level=0, inplace=True)
barcelona_countries_df

,country
0,es


In [43]:
#Transforming our city series into a city list to use with the api

bn_city = barcelona_cities_df['city']
bn_city
cities = bn_city.values.tolist()
cities

['badalona',
 'barcelona',
 'el prat de llobregat',
 'ripollet',
 'sant cugat del valles',
 'sant just desvern']

In [44]:
#Transforming our city series into a city list to use with the api

bn_country = barcelona_countries_df['country']
#nz_city
country = bn_country[0]


In [46]:
#Open weather information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"



#field_names
#empty lists for the fields of our future city weather csv
city_list = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country_list = []

for city in cities:
    
    #Get the city's number
    city_no = cities.index(city) + 1
    
    try:
        query_url = f"{url}appid={weather_api_key}&q={city},{country}&units={units}"
        
        

        # Get weather data
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()

        #Put the city's weather into lists
        city_list.append(weather_json["name"])
        lat.append(weather_json["coord"]["lat"])
        lng.append(weather_json["coord"]["lon"])
        max_temp.append(weather_json["main"]["temp_max"])
        humidity.append(weather_json["main"]["humidity"])
        cloudiness.append(weather_json["clouds"]["all"])
        wind_speed.append(weather_json["wind"]["speed"])
        country_list.append(weather_json["sys"]["country"])
        
        print(f"Getting {city}\'s weather information(City #{city_no})." )
    except KeyError:
        print(f"Can't find {city} (City #{city_no}) .")

Getting badalona's weather information(City #1).
Getting barcelona's weather information(City #2).
Getting el prat de llobregat's weather information(City #3).
Getting ripollet's weather information(City #4).
Getting sant cugat del valles's weather information(City #5).
Getting sant just desvern's weather information(City #6).


In [47]:
#Putting city results into a dictionary
city_dict = {'City': city_list, 'Lat': lat, 'Lng': lng, "Max Temp": max_temp, "Humidity": humidity, "Cloudiness": cloudiness, "Wind Speed": wind_speed, "Country": country_list}


In [ ]:
#Loading Barcelona

In [48]:
#Appending city results to csv

zd = zip(*city_dict.values())
#, encoding='utf-8'

with open("weather_data/cities.csv", 'a', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',', lineterminator = '\n')
    #writer.writerow(city_dict.keys())
    writer.writerows(zd)


In [ ]:
#Dataframe of our city data
#city_df = pd.read_csv("weather_data/cities.csv")
#city_df.head()

In [ ]:
#Extracting Sydney data

In [61]:
sydney_df = pd.read_csv("airbb_data/sydney_aus_airbb.csv")

sydney = sydney_df[['latitude', 'longitude']]

In [62]:
sydney.head()

,latitude,longitude
0,-33.86934,151.22688
1,-33.86515,151.19190
2,-33.80093,151.26172
3,-33.87964,151.21680
4,-33.87888,151.21439


In [63]:
#Using the citipy package to find the nearest city to the lat and long provided
sydney['city'] = sydney.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).city_name, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [64]:
#Using the citipy package to find the nearest city to the lat and long provided
sydney['country'] = sydney.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).country_code, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [65]:
sydney.head()

,latitude,longitude,city,country
0,-33.86934,151.22688,sydney,au
1,-33.86515,151.19190,sydney,au
2,-33.80093,151.26172,sydney,au
3,-33.87964,151.21680,sydney,au
4,-33.87888,151.21439,sydney,au


In [66]:
#Grouping by city to remove duplicates

sydney_cities = sydney[['city']].groupby(['city'])
sydney_cities_df = sydney_cities.sum()
sydney_cities_df.reset_index(level=0, inplace=True)
sydney_cities_df

,city
0,lithgow
1,sydney
2,wollongong


In [67]:
#Grouping by city to remove duplicates

sydney_countries = sydney[['country']].groupby(['country'])
sydney_countries_df = sydney_countries.sum()
sydney_countries_df.reset_index(level=0, inplace=True)
sydney_countries_df

,country
0,au


In [68]:
#Transforming our city series into a city list to use with the api

sy_city = sydney_cities_df['city']
sy_city
cities = sy_city.values.tolist()
cities

['lithgow', 'sydney', 'wollongong']

In [69]:
#Transforming our city series into a city list to use with the api

sy_country = sydney_countries_df['country']
#nz_city
country = sy_country[0]


In [70]:
#Open weather information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"



#field_names
#empty lists for the fields of our future city weather csv
city_list = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country_list = []

for city in cities:
    
    #Get the city's number
    city_no = cities.index(city) + 1
    
    try:
        query_url = f"{url}appid={weather_api_key}&q={city},{country}&units={units}"
        
        

        # Get weather data
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()

        #Put the city's weather into lists
        city_list.append(weather_json["name"])
        lat.append(weather_json["coord"]["lat"])
        lng.append(weather_json["coord"]["lon"])
        max_temp.append(weather_json["main"]["temp_max"])
        humidity.append(weather_json["main"]["humidity"])
        cloudiness.append(weather_json["clouds"]["all"])
        wind_speed.append(weather_json["wind"]["speed"])
        country_list.append(weather_json["sys"]["country"])
        
        print(f"Getting {city}\'s weather information(City #{city_no})." )
    except KeyError:
        print(f"Can't find {city} (City #{city_no}) .")

Getting lithgow's weather information(City #1).
Getting sydney's weather information(City #2).
Getting wollongong's weather information(City #3).


In [71]:
#Putting city results into a dictionary
city_dict = {'City': city_list, 'Lat': lat, 'Lng': lng, "Max Temp": max_temp, "Humidity": humidity, "Cloudiness": cloudiness, "Wind Speed": wind_speed, "Country": country_list}


In [72]:
#Loading Sydney

In [73]:
#Appending city results to csv

zd = zip(*city_dict.values())
#, encoding='utf-8'

with open("weather_data/cities.csv", 'a', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',', lineterminator = '\n')
    #writer.writerow(city_dict.keys())
    writer.writerows(zd)


In [ ]:
#Extracting Thailand data

In [74]:
thailand_df = pd.read_csv("airbb_data/thailand.csv")

thailand = thailand_df[['latitude', 'longitude']]

In [75]:
thailand.head()

,latitude,longitude
0,13.76000,100.54269
1,13.73463,100.47540
2,13.66746,100.61540
3,13.80970,100.56527
4,13.75379,100.62422


In [76]:
#Using the citipy package to find the nearest city to the lat and long provided
thailand['city'] = thailand.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).city_name, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [77]:
#Using the citipy package to find the nearest city to the lat and long provided
thailand['country'] = thailand.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).country_code, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
thailand.head()

,latitude,longitude,city,country
0,13.76000,100.54269,bangkok,th
1,13.73463,100.47540,bangkok,th
2,13.66746,100.61540,samut prakan,th
3,13.80970,100.56527,nonthaburi,th
4,13.75379,100.62422,bangkok,th


In [79]:
#Grouping by city to remove duplicates

thailand_cities = thailand[['city']].groupby(['city'])
thailand_cities_df = thailand_cities.sum()
thailand_cities_df.reset_index(level=0, inplace=True)
thailand_cities_df

,city
0,bang sao thong
1,bang yai
2,bangkok
3,chachoengsao
4,krathum baen
5,lam luk ka
6,nonthaburi
7,pak kret
8,pathum thani
9,phra pradaeng


In [80]:
#Grouping by city to remove duplicates

thailand_countries = thailand[['country']].groupby(['country'])
thailand_countries_df = thailand_countries.sum()
thailand_countries_df.reset_index(level=0, inplace=True)
thailand_countries_df

,country
0,th


In [81]:
#Transforming our city series into a city list to use with the api

td_city = thailand_cities_df['city']
td_city
cities = td_city.values.tolist()
cities

['bang sao thong',
 'bang yai',
 'bangkok',
 'chachoengsao',
 'krathum baen',
 'lam luk ka',
 'nonthaburi',
 'pak kret',
 'pathum thani',
 'phra pradaeng',
 'sam phran',
 'samut prakan',
 'samut sakhon']

In [82]:
#Transforming our city series into a city list to use with the api

td_country = thailand_countries_df['country']
#nz_city
country = td_country[0]


In [83]:
#Open weather information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"



#field_names
#empty lists for the fields of our future city weather csv
city_list = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country_list = []

for city in cities:
    
    #Get the city's number
    city_no = cities.index(city) + 1
    
    try:
        query_url = f"{url}appid={weather_api_key}&q={city},{country}&units={units}"
        
        

        # Get weather data
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()

        #Put the city's weather into lists
        city_list.append(weather_json["name"])
        lat.append(weather_json["coord"]["lat"])
        lng.append(weather_json["coord"]["lon"])
        max_temp.append(weather_json["main"]["temp_max"])
        humidity.append(weather_json["main"]["humidity"])
        cloudiness.append(weather_json["clouds"]["all"])
        wind_speed.append(weather_json["wind"]["speed"])
        country_list.append(weather_json["sys"]["country"])
        
        print(f"Getting {city}\'s weather information(City #{city_no})." )
    except KeyError:
        print(f"Can't find {city} (City #{city_no}) .")

Getting bang sao thong's weather information(City #1).
Getting bang yai's weather information(City #2).
Getting bangkok's weather information(City #3).
Getting chachoengsao's weather information(City #4).
Getting krathum baen's weather information(City #5).
Getting lam luk ka's weather information(City #6).
Getting nonthaburi's weather information(City #7).
Getting pak kret's weather information(City #8).
Getting pathum thani's weather information(City #9).
Getting phra pradaeng's weather information(City #10).
Getting sam phran's weather information(City #11).
Getting samut prakan's weather information(City #12).
Getting samut sakhon's weather information(City #13).


In [84]:
#Putting city results into a dictionary
city_dict = {'City': city_list, 'Lat': lat, 'Lng': lng, "Max Temp": max_temp, "Humidity": humidity, "Cloudiness": cloudiness, "Wind Speed": wind_speed, "Country": country_list}


In [ ]:
#Loading Thailand

In [85]:
#Appending city results to csv

zd = zip(*city_dict.values())
#, encoding='utf-8'

with open("weather_data/cities.csv", 'a', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',', lineterminator = '\n')
    #writer.writerow(city_dict.keys())
    writer.writerows(zd)


In [ ]:
#Extracting Venice data

In [86]:
venice_df = pd.read_csv("airbb_data/venice-airbb.csv")

venice = venice_df[['latitude', 'longitude']]

In [87]:
venice.head()

,latitude,longitude
0,45.43823,12.32902
1,45.43971,12.32799
2,45.44049,12.32877
3,45.43221,12.33411
4,45.44754,12.32918


In [88]:
#Using the citipy package to find the nearest city to the lat and long provided
venice['city'] = venice.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).city_name, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [89]:
#Using the citipy package to find the nearest city to the lat and long provided
venice['country'] = venice.apply(lambda row: citipy.nearest_city(row.latitude, row.longitude).country_code, axis = 1)

C:\Users\Beverly\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [90]:
venice.head()

,latitude,longitude,city,country
0,45.43823,12.32902,venice,it
1,45.43971,12.32799,venice,it
2,45.44049,12.32877,venice,it
3,45.43221,12.33411,venice,it
4,45.44754,12.32918,venice,it


In [91]:
#Grouping by city to remove duplicates

venice_cities = venice[['city']].groupby(['city'])
venice_cities_df = venice_cities.sum()
venice_cities_df.reset_index(level=0, inplace=True)
venice_cities_df

,city
0,chioggia
1,marcon
2,martellago
3,mogliano veneto
4,spinea
5,venice


In [92]:
#Grouping by city to remove duplicates

venice_countries = venice[['country']].groupby(['country'])
venice_countries_df = venice_countries.sum()
venice_countries_df.reset_index(level=0, inplace=True)
venice_countries_df

,country
0,it


In [93]:
#Transforming our city series into a city list to use with the api

vn_city = venice_cities_df['city']
vn_city
cities = vn_city.values.tolist()
cities

['chioggia', 'marcon', 'martellago', 'mogliano veneto', 'spinea', 'venice']

In [94]:
#Transforming our city series into a city list to use with the api

vn_country = venice_countries_df['country']
#nz_city
country = vn_country[0]


In [95]:
#Open weather information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"



#field_names
#empty lists for the fields of our future city weather csv
city_list = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country_list = []

for city in cities:
    
    #Get the city's number
    city_no = cities.index(city) + 1
    
    try:
        query_url = f"{url}appid={weather_api_key}&q={city},{country}&units={units}"
        
        

        # Get weather data
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()

        #Put the city's weather into lists
        city_list.append(weather_json["name"])
        lat.append(weather_json["coord"]["lat"])
        lng.append(weather_json["coord"]["lon"])
        max_temp.append(weather_json["main"]["temp_max"])
        humidity.append(weather_json["main"]["humidity"])
        cloudiness.append(weather_json["clouds"]["all"])
        wind_speed.append(weather_json["wind"]["speed"])
        country_list.append(weather_json["sys"]["country"])
        
        print(f"Getting {city}\'s weather information(City #{city_no})." )
    except KeyError:
        print(f"Can't find {city} (City #{city_no}) .")

Getting chioggia's weather information(City #1).
Getting marcon's weather information(City #2).
Getting martellago's weather information(City #3).
Getting mogliano veneto's weather information(City #4).
Getting spinea's weather information(City #5).
Getting venice's weather information(City #6).


In [96]:
#Putting city results into a dictionary
city_dict = {'City': city_list, 'Lat': lat, 'Lng': lng, "Max Temp": max_temp, "Humidity": humidity, "Cloudiness": cloudiness, "Wind Speed": wind_speed, "Country": country_list}


In [97]:
#Loading Venice

In [98]:
#Appending city results to csv

zd = zip(*city_dict.values())
#, encoding='utf-8'

with open("weather_data/cities.csv", 'a', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',', lineterminator = '\n')
    #writer.writerow(city_dict.keys())
    writer.writerows(zd)
